In [1]:
!pip install numpy matplotlib tensorflow

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [3]:
pip install kaggle


  Using cached bleach-6.3.0-py3-none-any.whl.metadata (31 kB)
  Using cached python_slugify-8.0.4-py2.py3-none-any.whl.metadata (8.5 kB)

   ------------- -------------------------- 2/6 [tqdm]
   ------------- -------------------------- 2/6 [tqdm]
   ------------- -------------------------- 2/6 [tqdm]
   ------------- -------------------------- 2/6 [tqdm]
   -------------------- ------------------- 3/6 [python-slugify]
   -------------------------- ------------- 4/6 [bleach]
   -------------------------- ------------- 4/6 [bleach]
   -------------------------- ------------- 4/6 [bleach]
   -------------------------- ------------- 4/6 [bleach]
   -------------------------- ------------- 4/6 [bleach]
   --------------------------------- ------ 5/6 [kaggle]
   --------------------------------- ------ 5/6 [kaggle]
   --------------------------------- ------ 5/6 [kaggle]
   --------------------------------- ------ 5/6 [kaggle]
   --------------------------------- ------ 5/6 [kaggle]
   ----

In [4]:

!kaggle datasets download -d anandkumarsahu09/cattle-breeds-dataset

Dataset URL: https://www.kaggle.com/datasets/anandkumarsahu09/cattle-breeds-dataset
License(s): CC0-1.0




  0%|          | 0.00/20.9M [00:00<?, ?B/s]
100%|██████████| 20.9M/20.9M [00:00<00:00, 948MB/s]


In [5]:
!tar -xf cattle-breeds-dataset.zip


In [6]:
datagen = ImageDataGenerator(
    rescale=1./255,          # normalize pixel values
    rotation_range=10,       # rotate images randomly
    width_shift_range=0.05,   # shift horizontally
    height_shift_range=0.05,  # shift vertically
    zoom_range=0.1,          # random zoomS
    horizontal_flip=True,    # flip horizontally
    validation_split=0.2     # 20% data for validation
)

# Training data
train_data = datagen.flow_from_directory(
    'cattle Breeds/',               # path to dataset folder
    target_size=(224,224),   # resize images
    batch_size=32,
    class_mode='categorical', # multi-class classification
    subset='training'
)

# Validation data
val_data = datagen.flow_from_directory(
    'cattle Breeds/',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 969 images belonging to 5 classes.
Found 239 images belonging to 5 classes.


In [7]:
print("Training samples:", train_data.samples)
print("Validation samples:", val_data.samples)
num_classes = train_data.num_classes
print("Number of breeds:", num_classes)

Training samples: 969
Validation samples: 239
Number of breeds: 5


In [8]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = True


In [9]:
for layer in base_model.layers[:100]:
    layer.trainable = False


In [10]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])


In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [12]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(           #If validation loss does not improve for 5 consecutive epochs, stop training automatically
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(       #If validation loss stops improving, reduce the learning rate.”
        monitor="val_loss",
        factor=0.2,
        patience=3,
        min_lr=1e-6
    )
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=callbacks
)


Epoch 1/15


31/31 [==============================] - 34s 941ms/step - loss: 1.4241 - accuracy: 0.4334 - val_loss: 1.1121 - val_accuracy: 0.6025 - lr: 1.0000e-04
Epoch 2/15
31/31 [==============================] - 15s 470ms/step - loss: 0.7693 - accuracy: 0.7162 - val_loss: 1.0945 - val_accuracy: 0.5858 - lr: 1.0000e-04
Epoch 3/15
31/31 [==============================] - 15s 495ms/step - loss: 0.4955 - accuracy: 0.8473 - val_loss: 1.0555 - val_accuracy: 0.6318 - lr: 1.0000e-04
Epoch 4/15
31/31 [==============================] - 15s 479ms/step - loss: 0.3426 - accuracy: 0.8875 - val_loss: 1.0222 - val_accuracy: 0.6569 - lr: 1.0000e-04
Epoch 5/15
31/31 [==============================] - 14s 448ms/step - loss: 0.2309 - accuracy: 0.9350 - val_loss: 1.0041 - val_accuracy: 0.6653 - lr: 1.0000e-04
Epoch 6/15
31/31 [==============================] - 14s 441ms/step - loss: 0.1653 - accuracy: 0.9577 - val_loss: 1.1175 - val_accuracy: 0.6485 - lr: 1.0000e-04
Epoch 7/15
31/31 [====================

In [13]:
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy after fine-tuning: {val_acc*100:.2f}%")


8/8 [==============================] - 2s 297ms/step - loss: 1.0334 - accuracy: 0.6569
Validation Accuracy after fine-tuning: 65.69%


In [14]:
y_true = val_data.classes
y_true


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [15]:
y_pred_probs = model.predict(val_data)
y_pred_probs


8/8 [==============================] - 3s 335ms/step


array([[9.4707645e-02, 3.9028670e-03, 8.6911052e-01, 1.2397018e-02,
        1.9881943e-02],
       [1.5684824e-03, 8.0452906e-03, 9.5427781e-01, 1.9817928e-02,
        1.6290540e-02],
       [5.3088088e-03, 8.0418130e-03, 9.6186977e-03, 9.6615857e-01,
        1.0872046e-02],
       ...,
       [2.4931712e-04, 1.1581321e-03, 9.9605054e-01, 2.3888163e-03,
        1.5320994e-04],
       [9.0247084e-04, 9.7502090e-02, 4.4858749e-03, 8.7406480e-01,
        2.3044685e-02],
       [1.4609046e-02, 6.5750025e-02, 3.7857473e-02, 7.1535844e-01,
        1.6642494e-01]], dtype=float32)

In [16]:
import numpy as np

y_pred = np.argmax(y_pred_probs, axis=1)
y_pred


array([2, 2, 3, 2, 0, 3, 4, 3, 2, 0, 0, 2, 2, 4, 2, 1, 2, 2, 3, 1, 4, 0,
       2, 4, 4, 0, 3, 0, 3, 4, 3, 3, 0, 3, 0, 2, 1, 3, 4, 3, 1, 3, 4, 3,
       2, 4, 2, 2, 3, 3, 3, 0, 3, 1, 1, 2, 4, 2, 0, 2, 1, 0, 3, 4, 0, 4,
       3, 3, 1, 4, 4, 2, 0, 2, 0, 1, 3, 3, 4, 2, 4, 3, 2, 4, 4, 0, 3, 3,
       4, 2, 2, 0, 2, 1, 2, 2, 4, 0, 1, 4, 3, 4, 1, 4, 3, 2, 0, 1, 2, 2,
       2, 4, 3, 1, 2, 4, 3, 0, 2, 1, 1, 4, 1, 3, 1, 3, 1, 3, 1, 3, 3, 3,
       3, 4, 4, 0, 4, 4, 2, 0, 2, 2, 4, 4, 1, 3, 0, 2, 3, 3, 0, 1, 2, 4,
       2, 1, 3, 3, 1, 4, 3, 3, 3, 0, 4, 2, 4, 3, 2, 2, 4, 4, 3, 4, 1, 4,
       4, 0, 4, 2, 3, 3, 0, 2, 3, 3, 4, 3, 2, 4, 2, 4, 3, 4, 1, 0, 2, 2,
       0, 3, 4, 2, 2, 4, 4, 2, 4, 4, 0, 3, 2, 2, 3, 0, 3, 2, 3, 2, 3, 2,
       3, 2, 0, 2, 1, 0, 3, 0, 3, 4, 2, 0, 3, 2, 2, 0, 2, 3, 3],
      dtype=int64)

In [17]:
!pip install scikit-learn
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)


[[ 9  4 14 16  9]
 [ 8  7 12  9 11]
 [ 5 10 10 13 12]
 [ 6  5 11 14 14]
 [ 7  1 14 12  6]]


In [18]:
print(val_data.class_indices)


{'Ayrshire cattle': 0, 'Brown Swiss cattle': 1, 'Holstein Friesian cattle': 2, 'Jersey cattle': 3, 'Red Dane cattle': 4}


In [19]:
from sklearn.metrics import classification_report

print(classification_report(
    y_true,
    y_pred,
    target_names=list(val_data.class_indices.keys())
))


                          precision    recall  f1-score   support

         Ayrshire cattle       0.26      0.17      0.21        52
      Brown Swiss cattle       0.26      0.15      0.19        47
Holstein Friesian cattle       0.16      0.20      0.18        50
           Jersey cattle       0.22      0.28      0.25        50
         Red Dane cattle       0.12      0.15      0.13        40

                accuracy                           0.19       239
               macro avg       0.20      0.19      0.19       239
            weighted avg       0.21      0.19      0.19       239



In [20]:
model.save("cattle_breed_model.keras")
